# 🦅 AVCI - Yüksek Oran Avcısı Eğitim Paneli v6.4.1 (God Mode Fix)
**Güncelleme:**
- **1000.0x (God Mode) Eklendi:** Artık 1000 katı gibi ekstrem hedefler için de optimizasyon ve eğitim yapabilirsin.
- Modüler yapı korundu (Her hedef için Optimizasyon + Eğitim ayrı hücrelerde).
---

In [ ]:
# 1. GitHub Deposunu Çek ve Kurulum Yap (Temiz Kurulum)
import os
import shutil

# 1. Kök dizine dön
try:
    os.chdir('/content')
except:
    pass

# 2. Varsa eski projeyi SİL
if os.path.exists('newproje'):
    print("Eski proje dosyaları temizleniyor...")
    shutil.rmtree('newproje')

# 3. Sıfırdan İndir
print("Projeyi GitHub'dan sıfırdan çekiyorum...")
!git clone https://github.com/onndd/newproje.git

# 4. Çalışma dizinine gir
if os.path.exists('newproje/avci'):
    os.chdir('newproje/avci')
    print(f"✅ Kurulum Başarılı. Çalışma Dizini: {os.getcwd()}")
    
    # --- VERİTABANI KURTARMA ---
    drive_db_path = '/content/drive/MyDrive/newproje/avci/jetx.db'
    local_db_path = 'jetx.db'
    
    if not os.path.exists('/content/drive'):
        try:
            from google.colab import drive
            drive.mount('/content/drive')
        except:
            pass

    if os.path.exists(drive_db_path):
        shutil.copy(drive_db_path, local_db_path)
        print("✅ Veritabanı güncellendi.")
    else:
        print("⚠️ UYARI: Drive'da 'jetx.db' bulunamadı!")
else:
    print("❌ HATA: İndirme başarısız oldu!")

!nvidia-smi
!pip install optuna streamlit matplotlib pandas tensorflow hmmlearn plotly lightgbm
print("Kurulum ve Hazırlık Tamamlandı!")

In [ ]:
# 2. Veri Yükleme ve Ayarlar
import optuna
import logging
import sys
import os

optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

try:
    # Modülleri içe aktar (Absolute Import)
    import train_avci
    import models_avci
    import model_ae_avci
    import model_hmm_avci
    
    # Fonksiyonları kısayol olarak al
    load_and_prep = train_avci.load_and_prep
    visualize_performance = train_avci.visualize_performance
    optimize_target = train_avci.optimize_target
    train_target_final = train_avci.train_target_final
    
    print("✅ Modüller başarıyla yüklendi!")
except ImportError as e:
    print(f"❌ HATA: Modül import hatası. Detay: {e}")

if 'load_and_prep' in locals():
    df = load_and_prep(limit=100000)
    print(f"Veri Hazır: {len(df)} satır.")

## 🕵️‍♂️ Bölüm 1: İstihbarat & Komuta

In [ ]:
if 'model_ae_avci' in locals():
    ae = model_ae_avci.train_autoencoder(df)

In [ ]:
if 'model_hmm_avci' in locals():
    hmm_model = model_hmm_avci.train_hmm(df)

## 🎯 Bölüm 2: Nişancı Eğitimi (Adım Adım)

In [ ]:
# 1.50x - OPTİMİZASYON 🛠️
TARGET = 1.5
EPOCHS = 30
study_15, params_15 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 1.50x - FİNAL EĞİTİM 🏃
model_15, _, _ = train_target_final(df, TARGET, params_15)
split_idx = int(len(df) * 0.85)
y_col = f'target_{str(TARGET).replace(".","_")}'
X_val = df.iloc[split_idx:][[c for c in df.columns if 'target' not in c and 'result' not in c and 'value' not in c and 'id' not in c]]
y_val = df[y_col].iloc[split_idx:]
visualize_performance(model_15, X_val, y_val, TARGET)

In [ ]:
# 3.00x - OPTİMİZASYON 🛠️
TARGET = 3.0
EPOCHS = 30
study_30, params_30 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 3.00x - FİNAL EĞİTİM 🏃
model_30, _, _ = train_target_final(df, TARGET, params_30)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_30, X_val, y_val, TARGET)

In [ ]:
# 5.00x - OPTİMİZASYON 🛠️
TARGET = 5.0
EPOCHS = 30
study_50, params_50 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 5.00x - FİNAL EĞİTİM 🏃
model_50, _, _ = train_target_final(df, TARGET, params_50)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_50, X_val, y_val, TARGET)

In [ ]:
# 10.0x - OPTİMİZASYON 🛠️
TARGET = 10.0
EPOCHS = 30
study_100, params_100 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 10.0x - FİNAL EĞİTİM 🏃
model_100, _, _ = train_target_final(df, TARGET, params_100)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_100, X_val, y_val, TARGET)

In [ ]:
# 100.0x - OPTİMİZASYON (Avcı Modu) 🛠️
TARGET = 100.0
EPOCHS = 50
study_1000, params_1000 = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 100.0x - FİNAL EĞİTİM 🏃
model_1000, _, _ = train_target_final(df, TARGET, params_1000)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_1000, X_val, y_val, TARGET)

In [ ]:
# 1000.0x - OPTİMİZASYON (God Mode) 🛠️
TARGET = 1000.0
EPOCHS = 50
study_god, params_god = optimize_target(df, TARGET, epochs=EPOCHS)

In [ ]:
# 1000.0x - FİNAL EĞİTİM 🏃
model_god, _, _ = train_target_final(df, TARGET, params_god)
y_val = df[f'target_{str(TARGET).replace(".","_")}'].iloc[split_idx:]
visualize_performance(model_god, X_val, y_val, TARGET)

## 💾 Bölüm 3: Modelleri Kaydet ve İndir

In [ ]:
# Tüm Modelleri Ziple
import os
if not os.path.exists('models'):
    os.makedirs('models')
    
!zip -r avci_models.zip models/

# İndir
try:
    from google.colab import files
    files.download('avci_models.zip')
except Exception as e:
    print(f"Hata: {e}")

if os.path.exists('/content/drive/MyDrive'):
    !cp avci_models.zip /content/drive/MyDrive/avci_backup_latest.zip
    print("Yedek Drive'a kopyalandı.")